<a href="https://colab.research.google.com/github/inactdev/CS668/blob/master/Deep_Value_Funnel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download and setup the libraries

In [ ]:
!pip install sec-edgar-downloader beautifulsoup4

import transformers
import pandas
import numpy
from bs4 import BeautifulSoup

# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import os
import json

def save_data_to_file(sections, path, name = None):
  if os.path.exists(path):
    # Open the file in write mode ('w')
    file_path = path
    if name is not None:
      file_path = os.path.join(path, name)
    print(file_path)
    with open(file_path, 'w') as file:
        # Use json.dump() to write the dictionary to the file
        json.dump(sections, file, indent=4) # indent for better readability

    print(f"Dictionary saved to {file_path}")
  else:
      print(f"Error: The path '{path}' does not exist.")

In [ ]:
import json

def load_data_from_file(file_path):
  # To load the dictionary back from the file
  try:
      with open(file_path, 'r') as file:
          data = json.load(file)
      return data
  except FileNotFoundError:
      print

In [ ]:
poor_performing_equity_ids = ["APA", "CCL", "VTRS", "KEY"]
well_performing_equity_ids = ["NVDA", "AMD", "TPL", "BLDR", "AXON", "AVGO", "ANET"]
avg_performing_equity_ids = ["JNJ", "PG", "KO", "PEP", "WMT", "HD", "LOW", "MCD", "UNH", "JPM", "BAC", "V", "MA", "XOM", "CVX", "PFE", "MRK", "INTC", "CSCO", "VZ"]

Download Filings to Google Drive

In [ ]:
from sec_edgar_downloader import Downloader

filepath = "/content/drive/My Drive/Pace/SEC Filings"
dl = Downloader("Ari Perez", "ap79624n@pace.edu", filepath)  # Required for SEC compliance

for equity_id in poor_performing_equity_ids:
  dl.get("10-K", equity_id, limit=10, before="2022-01-01") # Getting before 2022 so that we have 3 full years to check for doubles

for equity_id in well_performing_equity_ids:
  dl.get("10-K", equity_id, limit=10, before="2022-01-01") # Getting before 2022 so that we have 3 full years to check for doubles

for equity_id in avg_performing_equity_ids:
  dl.get("10-K", equity_id, limit=10, before="2022-01-01") # Getting before 2022 so that we have 3 full years to check for doubles

Download Stock Performance data to Google Drive

In [ ]:
!pip install alpha_vantage pandas

from google.colab import userdata
from alpha_vantage.timeseries import TimeSeries

api_key = userdata.get('ALPHA_VANTAGE_API_KEY')
root_path = "/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings"

def save_stock_data(symbol):
  path = f"{root_path}/{symbol}"
  name = "stock_prices.json"
  ts = TimeSeries(key=api_key, output_format="pandas")
  data, meta_data = ts.get_daily(symbol, outputsize="full")

  if not data.empty:
      print(data.head())
      save_data_to_file(data.to_json(), path, name)

for equity_id in poor_performing_equity_ids:
  save_stock_data(equity_id)

for equity_id in well_performing_equity_ids:
  save_stock_data(equity_id)

for equity_id in avg_performing_equity_ids:
  save_stock_data(equity_id)

Pull filenames for all SEC filings

In [ ]:
import os

rootdir = '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings'
all_filings = []

for subdir, dirs, files in os.walk(rootdir):
  for file in files:
    if "full-submission" in file:
      all_filings.append(os.path.join(subdir, file))

print(all_filings)

Pull filenames for SEC filings that have yet to be downloaded



In [ ]:
import os

rootdir = '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings'
filings = []

for subdir, dirs, files in os.walk(rootdir):
  for file in files:
    if "full-submission" in file and "sections.json" not in files:
      filings.append(os.path.join(subdir, file))

print(filings)

['/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/C/10-K/0000831001-21-000042/full-submission.txt', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/C/10-K/0000831001-20-000031/full-submission.txt', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/C/10-K/0000831001-19-000027/full-submission.txt', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/C/10-K/0000831001-18-000040/full-submission.txt', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/C/10-K/0000831001-17-000038/full-submission.txt', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/C/10-K/0000831001-16-000235/full-submission.txt', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/C/10-K/0000831001-15-000043/full-submission.txt', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/C/10-K/0001104659-14-015152/full-submission.txt', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/C/10-K/0001206774-13-000852/full-submission.txt', '/content

Pull filenames for Stock Prices

In [ ]:
import os

rootdir = '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings'
stock_price_files = []

for subdir, dirs, files in os.walk(rootdir):
  for file in files:
    if "stock_prices" in file:
      stock_price_files.append(os.path.join(subdir, file))

print(stock_price_files)

['/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/WBA/stock_prices.json', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/PARA/stock_prices.json', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/CCL/stock_prices.json', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/VTRS/stock_prices.json', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/KEY/stock_prices.json', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/NVDA/stock_prices.json', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/AMD/stock_prices.json', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/TPL/stock_prices.json', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/BLDR/stock_prices.json', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/AXON/stock_prices.json', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/AVGO/stock_prices.json', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/ANET/stoc

Pull filenames for SEC Filings Data Sections

In [ ]:
import os

rootdir = '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings'
filing_data_sections = []

for subdir, dirs, files in os.walk(rootdir):
  for file in files:
    if "sections.json" in file:
      filing_data_sections.append(os.path.join(subdir, file))

print(filing_data_sections)

['/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/WBA/10-K/0001618921-21-000085/sections.json', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/WBA/10-K/0001618921-20-000082/sections.json', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/WBA/10-K/0001618921-19-000069/sections.json', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/WBA/10-K/0001628280-18-012472/sections.json', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/WBA/10-K/0001618921-17-000069/sections.json', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/WBA/10-K/0001140361-16-083198/sections.json', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/WBA/10-K/0001140361-15-038791/sections.json', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/PARA/10-K/0000813828-21-000005/sections.json', '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/PARA/10-K/0000813828-20-000013/sections.json', '/content/drive/My Drive/Pace/SEC Filings/s

Preprocess Filings Methods

In [ ]:
import re

def find_item_title(text):
    match = re.search(r'^item\b\s*[a-zA-Z0-9]+', text, re.IGNORECASE)
    if match:
        return match.group(0)
    return None

In [ ]:
from bs4 import BeautifulSoup
import pdb

def extract_sections(file_name):
    """
    Downloads a 10-K SEC filing from the given URL, extracts the text, and splits it into sections
    based on the index (table of contents).

    Args:
        url (str): URL of the 10-K filing (e.g., Tesla or Microsoft filing URL).

    Returns:
        list: List of dictionaries, each with 'title' and 'content' keys, or None if processing fails.
    """


    # Step 1: Parse the HTML
    with open(file_name, "r") as f:
      soup = BeautifulSoup(f, "html.parser")

    year_end = return_year_end(soup.get_text())

    # Step 2: Find the table of contents rows
    toc_rows = soup.find_all(lambda tag: tag.name in ['tr'] and ('item' in tag.text.lower() or 'signatures' in tag.text.lower()))

    if not toc_rows:
        print("Table of contents rows not found in the filing.")
        return None

    # Step 3: Extract section links from the table of contents rows
    sections = []

    for row in toc_rows:
      row_text = row.get_text().strip()
      title = find_item_title(row_text)

      if not title:
        title = 'Signatures'

      link = row.find('a', href=lambda x: x and x.startswith('#'))

      if link is None:
        continue

      anchor = link['href'][1:]  # Remove '#' to get the anchor name
      sections.append({'title': title.upper(), 'anchor': anchor})

    if not sections:
        print("No sections found in the table of contents.")
        return [], None # Return empty list and None for year_end

    # Step 4: Find all anchor tags in the document
    anchor_tags = {sec['anchor']: soup.find(id=sec['anchor']) for sec in sections if soup.find(id=sec['anchor'])}

    # Step 5: Extract content for each section
    for i in range(len(sections)):
        anchor = sections[i]['anchor']

        if anchor not in anchor_tags:
            print(f"Warning: Anchor '{anchor}' not found in document for section '{sections[i]['title']}'.")
            sections[i]['content'] = ""
            continue

        current_anchor = anchor_tags[anchor]

        # Get the next anchor tag, or None if it's the last section
        next_anchor = None

        for j in range(i + 1, len(sections)):
            if sections[j]['anchor'] in anchor_tags:
                next_anchor = anchor_tags[sections[j]['anchor']]
                break

        content = []
        # Iterate through all elements following the current anchor
        for element in current_anchor.next_elements:
            # Stop if we reach the next anchor
            if next_anchor and element is next_anchor:
                break
            # Collect text from string nodes
            if isinstance(element, str):
                text = element.strip()
                if text:  # Only add non-empty strings
                    content.append(text)

        # Store the joined text in the section
        sections[i]['content'] = ' '.join(content)

    year_end = return_year_end(soup.get_text())

    return sections, year_end

In [ ]:
def doubled_check(start_price, end_price):
  if start_price is not None and end_price is not None:
    if (start_price * 2) <= end_price:
      return "Yes"
    else:
      return "No"
  else:
    return "N/A"

Extract the filing data into sections

In [ ]:
# Right now this will only extract the data for files that have no been processed yet.
# To redo for all filings change "filings" to "all_filings"
for file in filings:
  print(f"\nProcessing {file}")
  # Assign the result to a single variable
  result = extract_sections(file)

  # Check if the result is not None before unpacking
  if result is not None:
    sections = result
    year = file.split("/")[9].split("-")[1]
    symbol = file.split("/")[7]
    folder = os.path.dirname(file)

    if sections:
        print(f"Found {len(sections)} sections for {symbol} - {year}")
        for sec in sections:
            print(f"\nSection: {sec['title']}")
            print(f"Content (last 100 chars): {sec['content'][-10:]}...")

        name = "sections.json"
        save_data_to_file(sections, folder, name)
    else:
        print("Failed to extract sections.")

Create Sentiment Analysis Data

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
model = AutoModelForSequenceClassification.from_pretrained("ProsusAI/finbert")

def get_sentiment_for_section(section, override = False):
  item = section['title']
  content = section['content']

  if 'Positive' in section and not override:
    print(f"Already processed {item}")
    return;

  if not content:
    print("No content")
    return;

  # # Only want the items in the filing
  if 'item' in item.lower():
    item_number = item.split(" ")[-1]
    inputs = tokenizer(content, return_tensors="pt", truncation=True, max_length=512)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        sentiments = torch.softmax(logits, dim=1)
        sentiments_list = sentiments.tolist()[0]
        section["Positive"] = sentiments_list[0]
        section["Neutral"] = sentiments_list[1]
        section["Negative"] = sentiments_list[2]
  else:
    print(f"Not an item")
    return;

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def save_sentiment_analysis(section_file, override=False): # Make override true to reanalyze a section
  sections = load_data_from_file(section_file)
  results=[]

  for section in sections:
    get_sentiment_for_section(section, override)

  save_data_to_file(sections, section_file)

Process and save sentiment data

Add the sentiment data to the sections files

In [ ]:
import os

rootdir = '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings'
results = []

for subdir, dirs, files in os.walk(rootdir):
  for file in files:
    if len(subdir.split("/")) > 9:
      year = subdir.split("/")[9].split("-")[1]
      symbol = subdir.split("/")[7]

    if "sections" in file:
      section_file = os.path.join(subdir, file)
      filing_data = os.path.join(subdir, "full-submission.txt")

      print(f"\nProcessing {section_file}")

      save_sentiment_analysis(section_file) # Make override true to re-analyze a section
    else:
      print(f"No sections found for {symbol} - {year}")
      continue

Load data into dataframe

In [ ]:
import re

def return_year_end(text):
    # Define the regex
    pattern = r"(?i)year ended\s+([A-Za-z]+\s+\d+,\s+\d{4})"

    # Match it
    match = re.search(pattern, text)
    if match:
        return match.group(1)
    return None


In [ ]:
def get_value_or_none(df, row_index, column_name):
    """
    Retrieves a value from a DataFrame at the specified row and column.
    Returns None if the row or column does not exist.
    """
    try:
        return df.at[row_index, column_name]
    except KeyError:
        return None

In [ ]:
def get_stock_price(dataframe, date):
  price_column = '4. close'
  timestamp_delta = 1
  timestamp_int = int(date.timestamp()) # Convert to integer
  value = get_value_or_none(dataframe, str(timestamp_int) + "000", price_column)

  # Iterate by one day until a date is found or we've gone 7 days without finding a date
  while value is None and timestamp_delta < 7:
    timestamp_int = timestamp_int + timestamp_delta * 86400 # Move one day ahead
    timestamp_delta += 1
    value = get_value_or_none(dataframe, str(timestamp_int) + "000", price_column)

  if value is None:
    print("No stock price found")

  return value


In [ ]:
import os
import pandas
from bs4 import BeautifulSoup
from datetime import datetime
from dateutil.relativedelta import relativedelta

date_format = "%B %d, %Y"

rootdir = '/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings'
data = []

# section_file = "/content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/CSCO/10-K/0000858877-21-000013/sections.json"
# sections = load_data_from_file(section_file)
# section = sections[0]
# title = section.get('title')
# positive = section.get('Positive')
# negative = section.get('Negative')
# neutral = section.get('Neutral')

# if title and positive and negative and neutral:
#   data.append({
#       'symbol': symbol,
#       'year': year,
#       'title': title,
#       'positive': positive,
#       'negative': negative,
#       'neutral': neutral
#   })

for subdir, dirs, files in os.walk(rootdir):
  for file in files:
    if len(subdir.split("/")) > 9:
      year = subdir.split("/")[9].split("-")[1]
      symbol = subdir.split("/")[7]

    if "sections" in file:
      folder = os.path.join(rootdir, symbol)
      stock_prices_file = os.path.join(folder, "stock_prices.json")
      section_file = os.path.join(subdir, file)
      filing_data = os.path.join(subdir, "full-submission.txt")

      print(f"We're in {folder}")

      print(f"Getting year end from {filing_data}")

      with open(filing_data, "r") as f:
        soup = BeautifulSoup(f, "html.parser")

      year_end = return_year_end(soup.get_text())
      year_end_date = datetime.strptime(year_end, date_format)
      year_later = year_end_date + relativedelta(years=1)
      year_2_later = year_end_date + relativedelta(years=2)
      year_3_later = year_end_date + relativedelta(years=3)

      print(f"Loading stock data from {stock_prices_file}")

      stock_prices = load_data_from_file(stock_prices_file)
      stock_prices_dict = json.loads(stock_prices)
      df = pandas.DataFrame.from_dict(stock_prices_dict)

      print(f"Getting stock prices for {year_end_date}")

      year_end_value = get_stock_price(df, year_end_date)
      year_later_value = get_stock_price(df, year_later)
      year_2_later_value = get_stock_price(df, year_2_later)
      year_3_later_value = get_stock_price(df, year_3_later)

      print(f"Year End: {year_end_value}")
      print(f"Year Later: {year_later_value}")
      print(f"Year 2 Later: {year_2_later_value}")
      print(f"Year 3 Later: {year_3_later_value}")

      print(f"\nProcessing {section_file}")

      sections = load_data_from_file(section_file)

      for section in sections:
        title = section.get('title')
        positive = section.get('Positive')
        negative = section.get('Negative')
        neutral = section.get('Neutral')

        if title and positive and negative and neutral and year_end_value and year_later_value and year_2_later_value and year_3_later_value:
          doubled = doubled_check(year_end_value, year_later_value)
          year_2_doubled = doubled_check(year_end_value, year_2_later_value)
          year_3_doubled = doubled_check(year_end_value, year_3_later_value)

          print(f"Doubled: {doubled}")
          print(f"Year 2 Doubled: {year_2_doubled}")
          print(f"Year 3 Doubled: {year_3_doubled}")

          data.append({
            'symbol': symbol,
            'year': year,
            'title': title,
            'positive': positive,
            'negative': negative,
            'neutral': neutral,
            'doubled': doubled,
            'year_2_doubled': year_2_doubled,
            'year_3_doubled': year_3_doubled
          })


Streaming output truncated to the last 5000 lines.
Year 3 Later: 45.86

Processing /content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/AMD/10-K/0000002488-17-000043/sections.json
We're in /content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/AMD
Getting year end from /content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/AMD/10-K/0000002488-16-000111/full-submission.txt
Loading stock data from /content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/AMD/stock_prices.json
Getting stock prices for 2015-12-26 00:00:00
Year End: 3.0
Year Later: 12.07
Year 2 Later: 10.46
Year 3 Later: 17.9

Processing /content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/AMD/10-K/0000002488-16-000111/sections.json
We're in /content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/AMD
Getting year end from /content/drive/My Drive/Pace/SEC Filings/sec-edgar-filings/AMD/10-K/0001193125-15-054362/full-submission.txt
Loading stock data from /content/drive/My Drive/Pace/SEC Filings/sec-edg

Save data to CSV

In [ ]:
df = pandas.DataFrame(data)
df.to_csv('/content/drive/My Drive/Pace/SEC Filings/dataset.csv', index=False)
display(df)

,symbol,year,title,positive,negative,neutral,doubled,year_2_doubled,year_3_doubled
0,CCL,21,ITEM 1,0.072504,0.906192,0.021303,No,No,No
1,CCL,21,ITEM 1A,0.026685,0.569043,0.404272,No,No,No
2,CCL,21,ITEM 1B,0.020724,0.793103,0.186172,No,No,No
3,CCL,21,ITEM 2,0.023164,0.954781,0.022054,No,No,No
4,CCL,21,ITEM 3,0.028397,0.898539,0.073064,No,No,No
...,...,...,...,...,...,...,...,...,...
1087,VZ,20,ITEM 12,0.018398,0.937989,0.043613,No,No,No
1088,VZ,20,ITEM 13,0.029822,0.949840,0.020338,No,No,No
1089,VZ,20,ITEM 14,0.020896,0.948345,0.030760,No,No,No
1090,VZ,20,ITEM 15,0.035542,0.942581,0.021878,No,No,No


Clean up dataset

In [ ]:
import re

# Valid titles
valid_titles = {
    'ITEM 1', 'ITEM 1A', 'ITEM 1B', 'ITEM 1C', 'ITEM 2', 'ITEM 3', 'ITEM 4', 'ITEM 5',
    'ITEM 6', 'ITEM 7', 'ITEM 7A', 'ITEM 8', 'ITEM 9', 'ITEM 9A', 'ITEM 9B', 'ITEM 9C',
    'ITEM 10', 'ITEM 11', 'ITEM 12', 'ITEM 13', 'ITEM 14', 'ITEM 15', 'ITEM 16'
}

def parse_title(title):
    match = re.match(r'ITEM (\d{1,2}[A-Z]?)', title)
    if match:
        full_capture = match.group(1)  # e.g., '2P', '1A', '10D'
        # Check full capture first
        full_title = 'ITEM ' + full_capture
        if full_title in valid_titles:
            return full_title
        # Extract numeric part and check
        numeric_part = re.match(r'\d+', full_capture).group()  # e.g., '2', '1', '10'
        numeric_title = 'ITEM ' + numeric_part
        if numeric_title in valid_titles:
            return numeric_title
    return title  # Return original if no valid match

# Clean the 'title' column by replacing all whitespace sequences with a single space and stripping edges
df['title'] = df['title'].apply(lambda x: re.sub(r'\s+', ' ', x.strip()))
df['title'] = df['title'].apply(parse_title)

display(df.head())
unique_values = df['title'].unique()
print(unique_values)

print(len(df))

# REMOVE ALL ROWS THAT DON'T HAVE A VALID TITLE
df = df[df['title'].isin(valid_titles)]

display(df.head())
unique_values = df['title'].unique()
print(unique_values)

print(len(df))

,symbol,year,title,positive,negative,neutral,doubled,year_2_doubled,year_3_doubled
0,CCL,21,ITEM 1,0.072504,0.906192,0.021303,No,No,No
1,CCL,21,ITEM 1A,0.026685,0.569043,0.404272,No,No,No
2,CCL,21,ITEM 1B,0.020724,0.793103,0.186172,No,No,No
3,CCL,21,ITEM 2,0.023164,0.954781,0.022054,No,No,No
4,CCL,21,ITEM 3,0.028397,0.898539,0.073064,No,No,No


['ITEM 1' 'ITEM 1A' 'ITEM 1B' 'ITEM 2' 'ITEM 3' 'ITEM 4' 'ITEM 5' 'ITEM 6'
 'ITEM 7' 'ITEM 7A' 'ITEM 8' 'ITEM 9' 'ITEM 9A' 'ITEM 9B' 'ITEM 10'
 'ITEM 11' 'ITEM 12' 'ITEM 13' 'ITEM 14' 'ITEM 15' 'ITEM 16' 'ITEM 9C'
 'ITEM X']
1092


,symbol,year,title,positive,negative,neutral,doubled,year_2_doubled,year_3_doubled
0,CCL,21,ITEM 1,0.072504,0.906192,0.021303,No,No,No
1,CCL,21,ITEM 1A,0.026685,0.569043,0.404272,No,No,No
2,CCL,21,ITEM 1B,0.020724,0.793103,0.186172,No,No,No
3,CCL,21,ITEM 2,0.023164,0.954781,0.022054,No,No,No
4,CCL,21,ITEM 3,0.028397,0.898539,0.073064,No,No,No


['ITEM 1' 'ITEM 1A' 'ITEM 1B' 'ITEM 2' 'ITEM 3' 'ITEM 4' 'ITEM 5' 'ITEM 6'
 'ITEM 7' 'ITEM 7A' 'ITEM 8' 'ITEM 9' 'ITEM 9A' 'ITEM 9B' 'ITEM 10'
 'ITEM 11' 'ITEM 12' 'ITEM 13' 'ITEM 14' 'ITEM 15' 'ITEM 16' 'ITEM 9C']
1091


In [ ]:
# SAVE
df.to_csv('/content/drive/My Drive/Pace/SEC Filings/dataset.csv', index=False)

Load and analyze the data

In [ ]:
sentiment_data = pandas.read_csv('/content/drive/My Drive/Pace/SEC Filings/dataset.csv')

display(sentiment_data.head())

,symbol,year,title,positive,negative,neutral,doubled,year_2_doubled,year_3_doubled
0,CCL,21,ITEM 1,0.072504,0.906192,0.021303,No,No,No
1,CCL,21,ITEM 1A,0.026685,0.569043,0.404272,No,No,No
2,CCL,21,ITEM 1B,0.020724,0.793103,0.186172,No,No,No
3,CCL,21,ITEM 2,0.023164,0.954781,0.022054,No,No,No
4,CCL,21,ITEM 3,0.028397,0.898539,0.073064,No,No,No


Convert Yes and No to Numeric

In [ ]:
# Convert 'doubled' to numeric
sentiment_data['doubled'] = sentiment_data['doubled'].map({'Yes': 1, 'No': 0})
sentiment_data['year_2_doubled'] = sentiment_data['year_2_doubled'].map({'Yes': 1, 'No': 0})
sentiment_data['year_3_doubled'] = sentiment_data['year_3_doubled'].map({'Yes': 1, 'No': 0})

display(sentiment_data.head())

,symbol,year,title,positive,negative,neutral,doubled,year_2_doubled,year_3_doubled
0,CCL,21,ITEM 1,0.072504,0.906192,0.021303,0,0,0
1,CCL,21,ITEM 1A,0.026685,0.569043,0.404272,0,0,0
2,CCL,21,ITEM 1B,0.020724,0.793103,0.186172,0,0,0
3,CCL,21,ITEM 2,0.023164,0.954781,0.022054,0,0,0
4,CCL,21,ITEM 3,0.028397,0.898539,0.073064,0,0,0


Aggregate each items's positive, negative and neutral sentiment by symbol and year using pivot table

In [ ]:
# Pivot for positive, negative, neutral, replacing any unfound item with 1 for neutral and 0s for positive and negative
pivot_df = sentiment_data.pivot_table(
    index=['symbol', 'year'],
    columns='title',
    values=['positive', 'negative', 'neutral']
)

# Create the new MultiIndex for columns with all titles
metrics = ['positive', 'negative', 'neutral']
new_columns = pandas.MultiIndex.from_product([metrics, valid_titles], names=['metric', 'title'])

# Reindex the pivot table to include all titles
pivot_df = pivot_df.reindex(columns=new_columns)

# Fill missing values: 0 for positive and negative, 1 for neutral
pivot_df.loc[:, ('positive', slice(None))] = pivot_df.loc[:, ('positive', slice(None))].fillna(0)
pivot_df.loc[:, ('negative', slice(None))] = pivot_df.loc[:, ('negative', slice(None))].fillna(0)
pivot_df.loc[:, ('neutral', slice(None))] = pivot_df.loc[:, ('neutral', slice(None))].fillna(1)

# Extract doubled values for each symbol-year pair and add them to the pivot table
doubled_values = sentiment_data.groupby(['symbol', 'year'])['doubled'].first()
year_2_doubled_values = sentiment_data.groupby(['symbol', 'year'])['year_2_doubled'].first()
year_3_doubled_values = sentiment_data.groupby(['symbol', 'year'])['year_3_doubled'].first()
pivot_df['doubled'] = doubled_values
pivot_df['year_2_doubled'] = year_2_doubled_values
pivot_df['year_3_doubled'] = year_3_doubled_values

# Make them integers
if pivot_df['doubled'].dtype == float:
    pivot_df['doubled'] = pivot_df['doubled'].fillna(0).astype(int)
if pivot_df['year_2_doubled'].dtype == float:
    pivot_df['year_2_doubled'] = pivot_df['year_2_doubled'].fillna(0).astype(int)
if pivot_df['year_3_doubled'].dtype == float:
    pivot_df['year_3_doubled'] = pivot_df['year_3_doubled'].fillna(0).astype(int)

# Display the result
print(pivot_df.iloc[0])

metric          title  
positive        ITEM 15    0.018085
                ITEM 1A    0.104820
                ITEM 12    0.019479
                ITEM 9     0.025803
                ITEM 4     0.021846
                             ...   
neutral         ITEM 6     0.441599
                ITEM 1B    0.173045
doubled                    0.000000
year_2_doubled             1.000000
year_3_doubled             0.000000
Name: (AMD, 20), Length: 72, dtype: float64


Flatten for easier use by the model

In [ ]:
pivot_df.columns = ['_'.join(map(str, col)).strip() for col in pivot_df.columns.values]

print(pivot_df.head())
print(len(pivot_df))

             positive_ITEM 15  positive_ITEM 1A  positive_ITEM 12  \
symbol year                                                         
AMD    20            0.018085          0.104820          0.019479   
       21            0.018490          0.022443          0.019470   
ANET   20            0.016950          0.047179          0.021617   
       21            0.016996          0.021107          0.021874   
AVGO   19            0.019335          0.025949          0.021665   

             positive_ITEM 9  positive_ITEM 4  positive_ITEM 2  \
symbol year                                                      
AMD    20           0.025803         0.021846         0.058774   
       21           0.025803         0.022211         0.052532   
ANET   20           0.025803         0.023227         0.562126   
       21           0.025803         0.023414         0.221207   
AVGO   19           0.020111         0.023104         0.023007   

             positive_ITEM 7  positive_ITEM 14  posit

In [ ]:
counts = pivot_df['doubled_'].value_counts()
print(counts)

counts = pivot_df['year_2_doubled_'].value_counts()
print(counts)

counts = pivot_df['year_3_doubled_'].value_counts()
print(counts)

doubled_
0    52
1     2
Name: count, dtype: int64
year_2_doubled_
0    49
1     5
Name: count, dtype: int64
year_3_doubled_
0    48
1     6
Name: count, dtype: int64


Create a model

In [ ]:
# USED year_3_doubled since it has the most doubles

X = pivot_df.drop(columns=['doubled_', 'year_2_doubled_', 'year_3_doubled_'])
y = pivot_df['year_3_doubled_'] # Change this to predict another year

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix

def result_metrics(model, X_test, y_test):
  y_pred = model.predict(X_test)
  print("Classification Report:")
  print(classification_report(y_test, y_pred))
  print("\nConfusion Matrix:")
  print(confusion_matrix(y_test, y_pred))

Linear Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(class_weight='balanced', max_iter=1000)  # Increase max_iter if convergence issues
model.fit(X_train, y_train)
result_metrics(model, X_test, y_test)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.75      0.86         8
           1       0.60      1.00      0.75         3

    accuracy                           0.82        11
   macro avg       0.80      0.88      0.80        11
weighted avg       0.89      0.82      0.83        11


Confusion Matrix:
[[6 2]
 [0 3]]


Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X_train, y_train)
result_metrics(model, X_test, y_test)

Classification Report:
              precision    recall  f1-score   support

           0       0.73      1.00      0.84         8
           1       0.00      0.00      0.00         3

    accuracy                           0.73        11
   macro avg       0.36      0.50      0.42        11
weighted avg       0.53      0.73      0.61        11


Confusion Matrix:
[[8 0]
 [3 0]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


XGBoost

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(scale_pos_weight=(len(y_train[y_train==0]) / len(y_train[y_train==1])), use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)
result_metrics(model, X_test, y_test)

Classification Report:
              precision    recall  f1-score   support

           0       0.73      1.00      0.84         8
           1       0.00      0.00      0.00         3

    accuracy                           0.73        11
   macro avg       0.36      0.50      0.42        11
weighted avg       0.53      0.73      0.61        11


Confusion Matrix:
[[8 0]
 [3 0]]


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:55:58] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in 

Feature Elimination

In [ ]:
from sklearn.feature_selection import RFE
selector = RFE(model, n_features_to_select=10)
selector = selector.fit(X_train, y_train)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

Results with selected features

Linear Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(class_weight='balanced', max_iter=1000)  # Increase max_iter if convergence issues
model.fit(X_train, y_train)
result_metrics(model, X_test, y_test)

Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.75      0.86         8
           1       0.60      1.00      0.75         3

    accuracy                           0.82        11
   macro avg       0.80      0.88      0.80        11
weighted avg       0.89      0.82      0.83        11


Confusion Matrix:
[[6 2]
 [0 3]]


Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(class_weight='balanced', random_state=42)
model.fit(X_train, y_train)
result_metrics(model, X_test, y_test)

Classification Report:
              precision    recall  f1-score   support

           0       0.73      1.00      0.84         8
           1       0.00      0.00      0.00         3

    accuracy                           0.73        11
   macro avg       0.36      0.50      0.42        11
weighted avg       0.53      0.73      0.61        11


Confusion Matrix:
[[8 0]
 [3 0]]


/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


XGBoost

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier(scale_pos_weight=(len(y_train[y_train==0]) / len(y_train[y_train==1])), use_label_encoder=False, eval_metric='logloss')
model.fit(X_train, y_train)
result_metrics(model, X_test, y_test)

Classification Report:
              precision    recall  f1-score   support

           0       0.73      1.00      0.84         8
           1       0.00      0.00      0.00         3

    accuracy                           0.73        11
   macro avg       0.36      0.50      0.42        11
weighted avg       0.53      0.73      0.61        11


Confusion Matrix:
[[8 0]
 [3 0]]


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [14:58:15] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in 